In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from pydantic import BaseModel

In [ ]:
load_dotenv(override=True)

In [ ]:
# Tools
from langchain_community.utilities import GoogleSerperAPIWrapper
serper = GoogleSerperAPIWrapper()
tool_search = Tool(
    name="search",
    func=serper.run,
    description="Useful for when you need more information from an online search"
)

pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")

def pushover_notification(message):
    url = "https://api.pushover.net/1/messages.json"
    data = {
        "token": pushover_token,
        "user": pushover_user,
        "message": message,
    }
    requests.post(url, data=data)

tool_push = Tool(
    name="send_push_notification",
    func=pushover_notification,
    description="Useful for sending push notifications to your phone"
)
tools = [tool_search, tool_push]

In [ ]:
# Step1: Initialise the State
class State(BaseModel):
    messages: Annotated[list, add_messages]

# Step2: Start the Graph Builder
graph_builder = StateGraph(State)

# Step3: Create Nodes
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)
def chat_node(old_state: State):
    response = llm_with_tools.invoke(old_state.messages)
    return {"messages": [response]}

graph_builder.add_node("chat_node", chat_node)
graph_builder.add_node("tools", ToolNode(tools))

# Step4: Creat Edges
graph_builder.add_conditional_edges('chat_node', tools_condition, 'tools')
graph_builder.add_edge('tools', 'chat_node')
graph_builder.add_edge(START, 'chat_node')

# Step5: Create the Graph
# Now if we want to add memory to the graph we can do this by adding a checkpoint to the graph

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
# GRAPH Invoke
config = {"configurable": {"thread_id": "1"}}
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()

In [ ]:
# Gets the current state of the graph
graph.get_state(config)

In [ ]:
# List of states
list(graph.get_state_history(config))

In [ ]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "checkpoints.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)


In [ ]:
# Step1: Initialise the State
class State(BaseModel):
    messages: Annotated[list, add_messages]

# Step2: Start the Graph Builder
graph_builder = StateGraph(State)

# Step3: Create Nodes
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)
def chat_node(old_state: State):
    response = llm_with_tools.invoke(old_state.messages)
    return {"messages": [response]}

graph_builder.add_node("chat_node", chat_node)
graph_builder.add_node("tools", ToolNode(tools))

# Step4: Creat Edges
graph_builder.add_conditional_edges('chat_node', tools_condition, 'tools')
graph_builder.add_edge('tools', 'chat_node')
graph_builder.add_edge(START, 'chat_node')

# Step5: Create the Graph
# Now if we want to add memory to the graph we can do this by adding a checkpoint to the graph

graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "3"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()